In [11]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import load_model
import os
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import DBSCAN

from keras.optimizers import Adam

from full_view_model import fcn_model
from util_func import *
from full_view_train import *
from test_on_udacity_data import *
from convert_to_full_view_panorama import *

from keras.utils.generic_utils import get_custom_objects
get_custom_objects().update({"my_loss": my_loss})


In [2]:
# load trained model
model = load_model('./saved_model/fv_model_for_car_June_28_46.h5')

[<tf.Tensor 'gradients/split_1_grad/concat:0' shape=(?, 16, 320, 8) dtype=float32>, None, None]


In [4]:
data_dir = './data/training_didi_data/car_train_edited/'
list_of_view = list_of_data(data_dir)
print('len(list_of_view): ', len(list_of_view))
#list_of_view = ['./data/training_didi_data/car_train_edited/suburu_leading_front_left/view/view_281.npy',
#                './data/training_didi_data/car_train_edited/cmax_following_long/view/view_6631.npy']

len(list_of_view):  17977


In [5]:
list_of_lidar = [file.replace('view','lidar') for file in list_of_view]
list_of_gtbox = [file.replace('car_train_edited', 'car_train_gt_box_edited').replace('view','gt_boxes3d') 
                 for file in list_of_view]
print(list_of_lidar[0])
print(list_of_gtbox[0])

./data/training_didi_data/car_train_edited/suburu_driving_towards_it/lidar/lidar_196.npy
./data/training_didi_data/car_train_gt_box_edited/suburu_driving_towards_it/gt_boxes3d/gt_boxes3d_196.npy


### Predict and clustering predicted box

In [50]:
def one_box_clustering(boxes, eps = 1, min_samples = 3):
    # Extract the center from predicted boxes
    box_centers = np.mean(boxes[:,[0,2],:2], axis = 1)
    # Do clustering
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(box_centers)
    labels = db.labels_
       
    n_clusters = len(set(labels))
    n_points = [np.sum([labels == i]) for i in range(n_clusters) ]
    max_point_cluster = np.argmax(n_points)
    
    index = (labels == max_point_cluster)
    box = np.mean(boxes[index],axis = 0)
    return np.expand_dims(box, 0)
    
    
    

In [43]:
from sklearn.cluster import DBSCAN

def predict(model,lidar, cluster = True, seg_thres=0.5, 
            cluster_dist = 0.2, min_dist = 2, neigbor_thres = 5):
    
    test_view =  fv_cylindrical_projection_for_test(lidar)
    
    view = test_view[:,:,[5,2]].reshape(1,16,320,2)
    
    list_boxes = []

    test_view_reshape = test_view.reshape(-1,6)
    pred = model.predict(view)
    pred = pred[0].reshape(-1,8)
    
    thres_pred = pred[pred[:,0] > seg_thres]
    thres_view = test_view_reshape[pred[:,0] > seg_thres]
    
    
    num_boxes = len(thres_pred)
    if num_boxes == 0:
        return np.array([]), np.array([])
    boxes = np.zeros((num_boxes,8,3))
    
    for i in range(num_boxes):
        boxes[i,0] = thres_view[i,:3] - rotation(thres_view[i,3],thres_pred[i,1:4])
        boxes[i,6] = thres_view[i,:3] - rotation(thres_view[i,3],thres_pred[i,4:7])

        boxes[i,2,:2] = boxes[i,6,:2]
        boxes[i,2,2] = boxes[i,0,2]

        phi = thres_pred[i,-1]

        z = boxes[i,2] - boxes[i,0]
        boxes[i,1,0] = (np.cos(phi)*z[0] + np.sin(phi)*z[1])*np.cos(phi) + boxes[i,0,0]
        boxes[i,1,1] = (-np.sin(phi)*z[0] + np.cos(phi)*z[1])*np.cos(phi) + boxes[i,0,1]
        boxes[i,1,2] = boxes[i,0,2]

        boxes[i,3] = boxes[i,0] + boxes[i,2] - boxes[i,1]
        boxes[i,4] = boxes[i,0] + boxes[i,6] - boxes[i,2]
        boxes[i,5] = boxes[i,1] + boxes[i,4] - boxes[i,0]
        boxes[i,7] = boxes[i,4] + boxes[i,6] - boxes[i,5]
    list_boxes.append(boxes)

    boxes = np.concatenate(list_boxes, axis = 0)
    if not cluster:
        return boxes
    
    else:
        box_cluster = one_box_clustering(boxes)

    return boxes, box_cluster

In [51]:
# randomly choose a frame 
i = np.random.randint(len(list_of_lidar))
lidar = np.load(list_of_lidar[i])
gtbox = np.load(list_of_gtbox[i])

boxes, box_cluster = predict(model, lidar)

In [52]:
print(boxes.shape), print(box_cluster.shape)
n_boxes = len(boxes)
print('Number of predited boxes: {}'.format(n_boxes))
print('Number of boxes after clustering: {}'.format(len(box_cluster)))
print('Randomly display 10 boxes then display clustered boxes')
choosen_boxes = np.random.randint(n_boxes, size = 10)
viz_mayavi_with_labels(lidar, boxes[choosen_boxes])
viz_mayavi_with_labels(lidar, box_cluster)

(13, 8, 3)
(1, 8, 3)
Number of predited boxes: 13
Number of boxes after clustering: 1
Randomly display 10 boxes then display clustered boxes


### predict on test data

In [54]:
#test_dir = './data/test_cars/'
test_lidar = np.load('./data/training_didi_data/car_train_edited/suburu_driving_towards_it/lidar/lidar_140.npy')
#test_lidar = np.load('./data/test_cars/ford01/lidar_0.npy')
boxes, box_cluster = predict(model, test_lidar)
n_boxes = len(boxes)
print('Number of predited boxes: {}'.format(n_boxes))
print('Number of boxes after clustering: {}'.format(len(box_cluster)))
print('Randomly display 10 boxes then display clustered boxes')
choosen_boxes = np.random.randint(n_boxes, size = 10)
viz_mayavi_with_labels(test_lidar, boxes[choosen_boxes])
viz_mayavi_with_labels(test_lidar, box_cluster)


Number of predited boxes: 5
Number of boxes after clustering: 1
Randomly display 10 boxes then display clustered boxes


In [34]:
def predict_bags(model, test_dir, box_test_dir):
    '''
    This function predict all the test file and save predicted boxes in box_test_dir
    '''
    if not os.path.exists(box_test_dir):
        os.mkdir(box_test_dir)
    
    bags = os.listdir(test_dir)
    for bag in bags:
        box_dir = os.path.join(box_test_dir, bag)
        if not os.path.exists(box_dir):
            os.mkdir(box_dir)
        lidar_dir = os.path.join(test_dir, bag)
        for f in os.listdir(lidar_dir):
            lidar = np.load(os.path.join(lidar_dir,f))
            boxes, box_cluster = predict(model, lidar)
            f_box = f.replace('lidar', 'boxes')
            f_cluster = f.replace('lidar', 'clustered_box')
            box_file = os.path.join(box_dir, f_box)
            np.save(box_file, boxes)
            cluster_file = os.path.join(box_dir, f_cluster)
            np.save(cluster_file, box_cluster)
    

In [35]:
test_dir = './data/test_cars/'
box_test_dir = './data/box_test_cars/'
predict_bags(model, test_dir, box_test_dir)

#### Make top view test video

In [36]:
import cv2

ImportError: libpcre.so.1: cannot open shared object file: No such file or directory